In [1]:
import pandas as pd
import pathlib
from recipe_preprocessor import import_recipes
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex

Prepare model.

In [2]:
transformer_model: str = "paraphrase-MiniLM-L6-v2"

model: SentenceTransformer = SentenceTransformer(transformer_model)

Load data.

In [3]:
recipe_path = pathlib.Path("./data/recipes.csv")
recipes: pd.DataFrame = import_recipes("./data/recipes.csv")

In [70]:
ingredients = recipes["RecipeIngredientParts"].values

Create embeddings.

In [46]:
recipes["RecipeIngredientString"] = recipes["RecipeIngredientParts"].str.join(" ")

In [48]:
column_to_embed = "RecipeIngredientString"

## Generate embeddings and store in Annoy index

In [50]:
embeddings = model.encode(recipes[column_to_embed ].values)
embedding_indexes = zip(recipes[column_to_embed ].index, embeddings)
vec_size = 384

index = AnnoyIndex(vec_size, "angular")

for embed in embedding_indexes:
    index.add_item(embed[0], embed[1])
    
index.build(10)
index.save(f"testIndex{column_to_embed}.ann")

True

## Query Annoy index

Generate query embedding and execute query:

In [52]:

testEmbedding = model.encode("sugar flour egg")

vectors = [recipes["Name"].loc[i] for i in index.get_nns_by_vector(testEmbedding, 10)]

In [53]:
vectors

['Oven Baked Eggs',
 'Whole Wheat Buttermilk Pancakes (With Wheat Germ)',
 'Buttermilk Wheat Pancakes',
 'Whole Wheat Buttermilk Pancakes',
 'Peanut Butter Eggs',
 'Rice Flour Pancakes',
 'Flourless Peanut Butter Pancakes',
 'Easy Cornmeal Buttermilk Pancakes',
 'Recipe: Homemade Buttermilk Pancakes',
 'Healthy Low-Fat Whole Wheat Buttermilk Pancakes']